In [ ]:
import pandas as pd


In [39]:
df = pd.read_csv('articles_coref.csv')

In [40]:
print(df['coref_chains'][0])

[{'chainID': 161, 'mentions': [{'text': 'Delhi', 'sentence': 22, 'start': 12, 'end': 13}, {'text': 'Delhi', 'sentence': 10, 'start': 11, 'end': 12}, {'text': 'Delhi', 'sentence': 8, 'start': 16, 'end': 17}, {'text': 'Delhi', 'sentence': 10, 'start': 19, 'end': 20}], 'representative': 'Delhi'}, {'chainID': 102, 'mentions': [{'text': 'The AAP leaders', 'sentence': 13, 'start': 1, 'end': 4}, {'text': 'they', 'sentence': 13, 'start': 9, 'end': 10}], 'representative': 'The AAP leaders'}, {'chainID': 166, 'mentions': [{'text': 'the AAP ’s', 'sentence': 12, 'start': 19, 'end': 22}, {'text': 'the AAP ’s', 'sentence': 22, 'start': 29, 'end': 32}, {'text': 'the AAP ’s', 'sentence': 12, 'start': 12, 'end': 15}], 'representative': 'the AAP ’s'}, {'chainID': 73, 'mentions': [{'text': 'the crowd', 'sentence': 8, 'start': 7, 'end': 9}, {'text': 'us', 'sentence': 8, 'start': 27, 'end': 28}], 'representative': 'the crowd'}, {'chainID': 105, 'mentions': [{'text': 'Prime Minister Narendra Modi', 'sentenc

In [ ]:
df = df.iloc[:15]
sub_df = df[['article_id', 'coref_chains']]

In [16]:
sub_df.to_csv('articles_coref_sample.csv', index=False)

In [19]:
df = sub_df


In [ ]:
df['core_chains'] = None

In [31]:
df2 = pd.read_csv('coref_annotations.csv', encoding='ISO-8859-1')


In [42]:
df = df.join(df2[['core_chain_annotations']])


In [43]:
df.head()

,links,articles,titles,pos_tags,ner_tags,coref_chains,core_chain_annotations
0,https://www.thehindu.com/elections/delhi-assem...,Lauding the Union Budget presented by Finance ...,"\nModi attacks AAP, lauds ‘people’s Budget’ in...","[('Lauding', 'VERB'), ('the', 'DET'), ('Union'...","[('the Union Budget', 'ORG'), ('Finance', 'ORG...","[{'chainID': 161, 'mentions': [{'text': 'Delhi...","[\n {\n ""chainID"": 1,\n ""representative..."
1,https://www.thehindu.com/elections/delhi-assem...,The EVMs and VVPAT machines used in the Delhi ...,\nEVMs kept in strongrooms with three-tier sec...,"[('The', 'DET'), ('EVMs', 'PROPN'), ('and', 'C...","[('VVPAT', 'ORG'), ('the Delhi Assembly', 'ORG...","[{'chainID': 54, 'mentions': [{'text': 'Delhi ...","[\n {\n ""chainID"": 1,\n ""representative..."
2,https://www.thehindu.com/elections/delhi-assem...,A day before polling for the 70-member Delhi A...,"\nAtishi, relatives of Bidhuri booked for poll...","[('A', 'DET'), ('day', 'NOUN'), ('before', 'AD...","[('70', 'CARDINAL'), ('Delhi Assembly', 'PERSO...","[{'chainID': 98, 'mentions': [{'text': 'she', ...","[\n {\n ""chainID"": 1,\n ""representative..."
3,https://www.thehindu.com/elections/delhi-assem...,Sitting next to his stall on a packed afternoo...,"\nBeset with dip in profits, inadequate infras...","[('Sitting', 'VERB'), ('next', 'ADV'), ('to', ...","[('afternoon', 'TIME'), ('Ghazipur', 'PERSON')...","[{'chainID': 18, 'mentions': [{'text': 'his', ...","[\n {\n ""chainID"": 1,\n ""representative..."
4,https://www.thehindu.com/elections/delhi-assem...,As she drives her e-rickshaw through the stree...,"\nInflation, civic amenities, representation, ...","[('As', 'SCONJ'), ('she', 'PRON'), ('drives', ...","[('south Delhi’s Khanpur', 'GPE'), ('Savitri',...","[{'chainID': 97, 'mentions': [{'text': 'women'...","[\n {\n ""chainID"": 1,\n ""representative..."


In [ ]:
import re
import json
import ast
import pandas as pd
from fuzzywuzzy import process  

def normalize_text(text): 
    text = text.lower().strip()
    text = re.sub(r"[^\w\s']", "", text)  # Remove special characters except apostrophes
    text = text.replace("’", "'")  # Normalize apostrophes
    return text

def extract_coref_info(coref_data, is_json=True):
   
    coref_info = set()
    # If data is already a list, use it directly
    if isinstance(coref_data, list):
        coref_list = coref_data  
    elif isinstance(coref_data, str) and coref_data.strip():
        try:
            if coref_data.startswith("[") and coref_data.endswith("]"):
                coref_list = ast.literal_eval(coref_data)  
            else:
                coref_list = json.loads(coref_data)  # Parse JSON string
        except (json.JSONDecodeError, SyntaxError):
            print(f"Error: Failed to parse data. Sample: {coref_data[:200]}")
            return coref_info
    else:
        print("Warning: Data is neither a valid list nor a JSON string.")
        return coref_info

    # Extract and normalize mentions
    for chain in coref_list:
        mentions = {normalize_text(mention["text"]) for mention in chain.get("mentions", [])}
        coref_info.update(mentions)
    return coref_info

def evaluate_coref_annotations(df):
    """Evaluate coreference annotations using precision, recall, F1-score, and accuracy."""
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for _, row in df.iterrows():
        # Process coref_chains which are python lists and core_chain_annotations which are json strings
        coref_mentions = extract_coref_info(row["coref_chains"], is_json=False)
        annotations_mentions = extract_coref_info(row["core_chain_annotations"], is_json=True)

        # Debug prints for verification
        print(f"Coref Mentions: {coref_mentions}")
        print(f"Annotated Mentions: {annotations_mentions}")

        # Use fuzzy matching to with 80% similarity 
        matched_coref_mentions = set()
        for mention in annotations_mentions:
            result = process.extractOne(mention, coref_mentions, score_cutoff=80)
            if result is not None:
                best_match, score = result
                matched_coref_mentions.add(best_match)

        # Compute True Positives (TP), False Positives (FP), False Negatives (FN)
        tp = len(matched_coref_mentions)
        fp = len(coref_mentions - matched_coref_mentions)
        fn = len(annotations_mentions - matched_coref_mentions)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        accuracy_scores.append(accuracy)

    # Average scores across all rows
    metrics_result = {
        "Precision": sum(precision_scores) / len(precision_scores) if precision_scores else 0,
        "Recall": sum(recall_scores) / len(recall_scores) if recall_scores else 0,
        "F1-score": sum(f1_scores) / len(f1_scores) if f1_scores else 0,
        "Accuracy": sum(accuracy_scores) / len(accuracy_scores) if accuracy_scores else 0,
    }
    return metrics_result
evaluation_results = evaluate_coref_annotations(df) #save the evaluations metrics

In [ ]:
print("\nEvaluation Results:")
for metric, score in evaluation_results.items():
    print(f"{metric}: {score:.4f}")
    


Evaluation Results:
Precision: 0.4408
Recall: 0.4480
F1-score: 0.4183
Accuracy: 0.2728
